# Country names

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

# Autores

In [2]:
%%time

autores_df = pd.read_csv('autores5.csv', header=0)
autores_df = autores_df.fillna('')

CPU times: user 6.33 s, sys: 254 ms, total: 6.58 s
Wall time: 6.58 s


In [3]:
autores_df.shape

(3054360, 6)

In [4]:
autores_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,NOMBRE_LOCALIZADO
0,Un bicho extraño,2010,Spain,Mon Daporta,DESCONOCIDO,DESCONOCIDO
1,Un bicho extraño,2010,Spain,Mon Daporta,DESCONOCIDO,DESCONOCIDO
2,Quisiera tener ...,2010,Mexico,Giovanna Zoboli,DESCONOCIDO,DESCONOCIDO
3,Johannes Gutenberg,2010,Spain,Lluís Borràs Perelló,DESCONOCIDO,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,2010,Spain,Sonia Alins,DESCONOCIDO,DESCONOCIDO


## Limpiando datos

### Elimina desconocidos

In [5]:
autores_df = autores_df[
    (autores_df['AUTOR'] != 'DESCONOCIDO') &
    (autores_df['TITULO'] != 'DESCONOCIDO') &
    (autores_df['PAIS_AUTOR'] != 'DESCONOCIDO') &
    (autores_df['PAIS_EJEMPLAR'] != 'DESCONOCIDO') &
    (autores_df['NOMBRE_LOCALIZADO'] != 'DESCONOCIDO')
]

In [6]:
autores_df.shape

(786709, 6)

In [7]:
autores_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,NOMBRE_LOCALIZADO
10,Adivina en qué pais ...,2010,Spain,Martina Badstuber,Germany,Martina Badstuber
11,Adivina en qué pais ...,2010,Spain,Martina Badstuber,Germany,Martina Badstuber
12,Adivina en qué pais ...,2010,Spain,Martina Badstuber,Germany,Martina Badstuber
21,Galleta para perros,2008,Spain,Helen Cooper,United Kingdom,Helen Cooper
22,Galleta para perros,2008,Spain,Helen Cooper,United Kingdom,Helen Cooper


### Elimina autores mapeados al mismo nombre, manteniendo el original y/o el más prestado

In [8]:
autores_count_sr = autores_df['AUTOR'].value_counts()

In [9]:
autor_nombre_localizado_df = autores_df[['AUTOR', 'NOMBRE_LOCALIZADO']].drop_duplicates().set_index('NOMBRE_LOCALIZADO')
nombre_localizado_autor_sr = pd.Series(autor_nombre_localizado_df['AUTOR'], index=autor_nombre_localizado_df.index)   

In [10]:
%%time

autor_nombre_localizado_dict = {}

for nl in nombre_localizado_autor_sr.index:
    autor = nombre_localizado_autor_sr[nl]
    if isinstance(autor, str):
        autor_nombre_localizado_dict[autor] = nl
    else:
        candidatos = [(a, autores_count_sr[a]) for a in autor]
        ganador = sorted(candidatos, key=lambda x: x[1], reverse=True)[0][0]
        autor_nombre_localizado_dict[ganador] = nl
        if nl in autor.values and nl != ganador:
            autor_nombre_localizado_dict[nl] = nl

CPU times: user 18.9 s, sys: 213 µs, total: 18.9 s
Wall time: 18.9 s


In [11]:
%%time

def process_nombre_localizado(x):
    a = x['AUTOR']
    return autor_nombre_localizado_dict.get(a, 'DESCONOCIDO')

autores_df['NOMBRE_LOCALIZADO'] = autores_df.apply(process_nombre_localizado, axis=1)

CPU times: user 21.2 s, sys: 10.5 ms, total: 21.2 s
Wall time: 21.2 s


In [12]:
autores_df = autores_df[autores_df['NOMBRE_LOCALIZADO'] != 'DESCONOCIDO']

In [13]:
len(autores_df)

778165

##### Seleccionamos ejemplares únicos de una misma obra. Un ejemplar se identifica por su título, país de edición, fecha de publicación y autor (el país de origen del autor no influye).

In [14]:
autores_df.drop_duplicates(['TITULO', 'PAIS_EJEMPLAR', 'FECHA_PUB', 'AUTOR'], inplace=True)

In [15]:
autores_df.shape

(61864, 6)

In [16]:
autores_df.head()

,TITULO,FECHA_PUB,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,NOMBRE_LOCALIZADO
10,Adivina en qué pais ...,2010,Spain,Martina Badstuber,Germany,Martina Badstuber
21,Galleta para perros,2008,Spain,Helen Cooper,United Kingdom,Helen Cooper
56,Codrilo,2009,Spain,Roberto Aliaga Sánchez,Spain,Roberto Aliaga
58,"Félix, : el coleccionista de miedos",2009,Spain,Fina Casalderrey,Spain,Fina Casalderrey
67,En la laguna más profunda,2011,Colombia,Oscar Collazos,Colombia,Oscar Collazos


## Resultados

In [17]:
autores_df['TITULO'].nunique()

50877

In [18]:
autores_df['TITULO'].value_counts()

Obras completas                                                 146
Antología poética                                               111
Cuentos                                                          93
Antología                                                        72
Poemas                                                           69
Cuentos completos                                                63
Poesía                                                           57
Poesía completa                                                  50
Teatro                                                           41
Obras escogidas                                                  41
Obra poética                                                     37
Obras                                                            37
Poesías completas                                                36
María                                                            34
Poesías                                         

In [19]:
autores_df['PAIS_EJEMPLAR'].nunique()

117

In [20]:
autores_df['PAIS_EJEMPLAR'].value_counts()

Spain                              32224
Colombia                            8009
Argentina                           6227
Mexico                              5390
United States                       2371
France                              1336
England                             1273
Venezuela                            804
Chile                                668
New York (State)                     537
Cuba                                 467
Brazil                               366
Peru                                 286
Italy                                255
United Kingdom                       223
Ecuador                              194
Uruguay                              119
Germany                              116
Portugal                              93
Soviet Union                          65
Costa Rica                            53
Japan                                 50
California                            50
Massachusetts                         40
Puerto Rico     

In [21]:
autores_df['NOMBRE_LOCALIZADO'].nunique()

11444

In [22]:
autores_df['NOMBRE_LOCALIZADO'].value_counts()

William Shakespeare         381
Gabriel García Márquez      367
Jules Verne                 220
Jorge Luis Borges           216
Miguel de Cervantes         213
Isaac Asimov                206
Oscar Wilde                 194
Agatha Christie             178
Julio Cortázar              176
Charles Dickens             170
Franz Kafka                 169
Pablo Neruda                167
Mario Vargas Llosa          162
Edgar Allan Poe             154
Friedrich Nietzsche         153
Stefan Zweig                149
Octavio Paz                 147
Robert Louis Stevenson      141
Mario Benedetti             134
Federico García Lorca       133
Leonard Huxley              132
Hans Christian Andersen     132
Jean Paul                   131
Honoré de Balzac            129
Hermann Hesse               123
Aristóteles                 122
Arthur Conan Doyle          122
Carlos Fuentes              121
Mark Twain                  121
Thomas Mann                 121
                           ... 
Gutierre

In [23]:
autores_df['PAIS_AUTOR'].nunique()

211

In [24]:
autores_df['PAIS_AUTOR'].value_counts()

United States of America                9190
Spain                                   7201
United Kingdom                          6571
France                                  5835
Colombia                                3450
Germany                                 3238
Argentina                               2707
Italy                                   2481
Mexico                                  1843
Cuba                                    1027
Chile                                    923
Austria                                  910
Belgium                                  896
Ireland                                  836
Peru                                     740
Uruguay                                  732
Brazil                                   726
España                                   669
India                                    632
Russia                                   619
Poland                                   588
Canada                                   506
Venezuela 

# Mapping countries

In [25]:
countries = set(autores_df['PAIS_EJEMPLAR']).union(autores_df['PAIS_AUTOR'])

In [26]:
inv_mappings = defaultdict(set)
for a in countries:
    for b in countries:
        if a != b and a in b:
            inv_mappings[a].add(b)
inv_mappings = dict(inv_mappings)

mapped = set()
for s in inv_mappings.values():
    mapped = mapped.union(s)

unmapped = countries.difference(mapped)

In [27]:
inv_mappings

{'Brazil': {'Empire of Brazil'},
 'China': {'China (Republic : 1949- )', "People's Republic of China"},
 'Denmark': {'Kingdom of Denmark'},
 'Dominica': {'Dominican Republic', 'República Dominicana'},
 'Finland': {'Finlandia'},
 'Francia': {'Antiguo Régimen en Francia'},
 'Germany': {'Allied-occupied Germany',
  'East Germany',
  'Germany (East)',
  'Nazi Germany'},
 'Guinea': {'Equatorial Guinea'},
 'India': {'Indiana'},
 'Ireland': {'Northern Ireland',
  'United Kingdom of Great Britain and Ireland'},
 'Jersey': {'New Jersey'},
 'Montenegro': {'Serbia and Montenegro'},
 'Netherlands': {'Caribbean Netherlands', 'Kingdom of the Netherlands'},
 'Philippines': {'Captaincy General of the Philippines'},
 'Russia': {'Russia (Federation)', 'Russian Empire'},
 'Serbia': {'Serbia and Montenegro'},
 'South Africa': {'Union of South Africa'},
 'United Kingdom': {'United Kingdom Misc. Islands',
  'United Kingdom of Great Britain and Ireland'},
 'United States': {'United States of America'},
 'Vie

In [28]:
mappings = {}
for c1 in inv_mappings:
    for c2 in inv_mappings[c1]:
        mappings[c2] = c1

In [29]:
mappings

{'Allied-occupied Germany': 'Germany',
 'Antiguo Régimen en Francia': 'Francia',
 'Captaincy General of the Philippines': 'Philippines',
 'Caribbean Netherlands': 'Netherlands',
 'China (Republic : 1949- )': 'China',
 'Dominican Republic': 'Dominica',
 'East Germany': 'Germany',
 'Empire of Brazil': 'Brazil',
 'Equatorial Guinea': 'Guinea',
 'Finlandia': 'Finland',
 'Germany (East)': 'Germany',
 'Indiana': 'India',
 'Kingdom of Denmark': 'Denmark',
 'Kingdom of the Netherlands': 'Netherlands',
 'Nazi Germany': 'Germany',
 'New Jersey': 'Jersey',
 'Northern Ireland': 'Ireland',
 "People's Republic of China": 'China',
 'República Dominicana': 'Dominica',
 'Russia (Federation)': 'Russia',
 'Russian Empire': 'Russia',
 'Serbia and Montenegro': 'Montenegro',
 'Union of South Africa': 'South Africa',
 'United Kingdom Misc. Islands': 'United Kingdom',
 'United Kingdom of Great Britain and Ireland': 'United Kingdom',
 'United States of America': 'United States',
 'Vietnam, North': 'Vietnam'}

In [30]:
# Correct wrong mappings

# 'Allied-occupied Germany': 'Germany',
# 'Antiguo Régimen en Francia': 'Francia',
mappings['Antiguo Régimen en Francia'] = 'France'
# 'Captaincy General of the Philippines': 'Philippines',
# 'Caribbean Netherlands': 'Netherlands',
del(mappings['Caribbean Netherlands'])
# 'China (Republic : 1949- )': 'China',
# 'Dominican Republic': 'Dominica',
del(mappings['Dominican Republic'])
# 'East Germany': 'Germany',
# 'Empire of Brazil': 'Brazil',
# 'Equatorial Guinea': 'Guinea',
del(mappings['Equatorial Guinea'])
# 'Finlandia': 'Finland',
# 'Germany (East)': 'Germany',
# 'Indiana': 'India',
del(mappings['Indiana'])
# 'Kingdom of Denmark': 'Denmark',
# 'Kingdom of the Netherlands': 'Netherlands',
# 'Nazi Germany': 'Germany',
# 'New Jersey': 'Jersey',
del(mappings['New Jersey'])
# 'Northern Ireland': 'Ireland',
del(mappings['Northern Ireland'])
# "People's Republic of China": 'China',
# 'República Dominicana': 'Dominica',
mappings['República Dominicana'] = 'Dominican Republic'
# 'Russia (Federation)': 'Russia',
# 'Russian Empire': 'Russia',
# 'Serbia and Montenegro': 'Serbia',
# 'Union of South Africa': 'South Africa',
# 'United Kingdom Misc. Islands': 'United Kingdom',
# 'United Kingdom of Great Britain and Ireland': 'Ireland',
mappings['United Kingdom of Great Britain and Ireland'] = 'United Kingdom'
# 'United States of America': 'United States',
# 'Vietnam, North': 'Vietnam',

In [31]:
unmapped

{'Afghanistan',
 'Alabama',
 'Albania',
 'Alemania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua Atenas',
 'Antigua and Barbuda',
 'Argentina',
 'Arizona',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Benin',
 'Bielorrusia',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Brasil',
 'Brazil',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Bélgica',
 'California',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Canadá',
 'Canal Zone',
 'Cape Verde',
 'Chile',
 'China',
 'Colombia',
 'Colorado',
 'Congo Belga',
 'Connecticut',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 'Czechoslovakia',
 "Côte d'Ivoire",
 'Denmark',
 'Dinamarca',
 'District of Columbia',
 'Dominica',
 'Ecuador',
 'Egipto',
 'Egypt',
 'El Salvador',
 'England',
 'Eritrea',
 'España',
 'Estados Unidos',
 'Estonia',
 'Ethiopia',
 'Finland',
 'Florida',
 'France',
 'Francia',
 'French Guiana',
 'Georgia',
 'Germany',
 'Ghana',
 'Grecia',
 'Greece',
 '

In [32]:
# Correct unmapped

# 'Afghanistan',
# 'Alabama',
# 'Albania',
# 'Alemania',
mappings['Alemania'] = 'Germany'
# 'Algeria',
# 'Andorra',
# 'Angola',
# 'Antigua Atenas',
mappings['Antigua Atenas'] = 'Greece'
# 'Antigua and Barbuda',
# 'Argentina',
# 'Arizona',
# 'Aruba',
# 'Australia',
# 'Austria',
# 'Azerbaijan',
# 'Bangladesh',
# 'Barbados',
# 'Belarus',
# 'Belgium',
# 'Benin',
# 'Bielorrusia',
mappings['Bielorrusia'] = 'Belarus'
# 'Bolivia',
# 'Bosnia and Herzegovina',
# 'Brasil',
mappings['Brasil'] = 'Brazil'
# 'Brazil',
# 'Bulgaria',
# 'Burkina Faso',
# 'Burma',
# 'Bélgica',
mappings['Bélgica'] = 'Belgium'
# 'California',
# 'Cambodia',
# 'Cameroon',
# 'Canada',
# 'Canadá',
mappings['Canadá'] = 'Canada'
# 'Canal Zone',
# 'Cape Verde',
# 'Chile',
# 'China',
# 'Colombia',
# 'Colorado',
# 'Congo Belga',
mappings['Congo Belga'] = 'Belgian Congo'
# 'Connecticut',
# 'Costa Rica',
# 'Croatia',
# 'Cuba',
# 'Curaçao',
# 'Cyprus',
# 'Czech Republic',
# 'Czechoslovakia',
# "Côte d'Ivoire",
mappings["Côte d'Ivoire"] = 'Ivory Coast'
# 'Denmark',
# 'Dinamarca',
mappings['Dinamarca'] = 'Denmark'
# 'District of Columbia',
# 'Dominica',
# 'Ecuador',
# 'Egipto',
mappings['Egipto'] = 'Egypt'
# 'Egypt',
# 'El Salvador',
# 'England',
# 'Eritrea',
# 'España',
mappings['España'] = 'Spain'
# 'Estados Unidos',
mappings['Estados Unidos'] = 'United States'
# 'Estonia',
# 'Ethiopia',
# 'Finland',
# 'Florida',
# 'France',
# 'Francia',
mappings['France'] = 'Francia'
# 'French Guiana',
# 'Georgia',
# 'Germany',
# 'Ghana',
# 'Grecia',
mappings['Grecia'] = 'Greece'
# 'Greece',
# 'Guatemala',
# 'Guinea',
# 'Guyana',
# 'Haiti',
# 'Honduras',
# 'Hong Kong',
# 'Hungary',
# 'Hungría',
mappings['Hungría'] = 'Hungary'
# 'Iceland',
# 'Illinois',
# 'Imperio Otomano',
mappings['Imperio Otomano'] = 'Ottoman Empire'
# 'Imperio otomano',
mappings['Imperio otomano'] = 'Ottoman Empire'
# 'Imperio romano',
mappings['Imperio romano'] = 'Italy'
# 'India',
# 'Indonesia',
# 'Iran',
# 'Iraq',
# 'Ireland',
# 'Irlanda',
mappings['Irlanda'] = 'Ireland'
# 'Irán',
mappings['Irán'] = 'Iran'
# 'Israel',
# 'Italia',
mappings['Italia'] = 'Italy'
# 'Italy',
# 'Jamaica',
# 'Japan',
# 'Japón',
mappings['Japón'] = 'Japan'
# 'Jersey',
# 'Jordan',
# 'Kentucky',
# 'Kenya',
# 'Kingdom of Galicia and Lodomeria',
# 'Kiribati',
# 'Korea (South)',
mappings['Korea (South)'] = 'South Korea'
# 'Kosovo',
# 'Kuwait',
# 'Latvia',
# 'Lebanon',
# 'Letonia',
mappings['Letonia'] = 'Latvia'
# 'Libia',
mappings['Libia'] = 'Libya'
# 'Libya',
# 'Lithuania',
# 'Lituania',
mappings['Lituania'] = 'Lithuania'
# 'Louisiana',
# 'Luxembourg',
# 'Luxemburgo',
mappings['Luxemburgo'] = 'Luxembourg'
# 'Líbano',
mappings['Líbano'] = 'Lebanon'
# 'Madagascar',
# 'Maine',
# 'Malawi',
# 'Malaysia',
# 'Mali',
# 'Malta',
# 'Mandato británico de Palestina',
mappings['Mandato británico de Palestina'] = 'Palestine'
# 'Mandatory Palestine',
mappings['Mandatory Palestine'] = 'Palestine'
# 'Marruecos',
mappings['Marruecos'] = 'Morocco'
# 'Maryland',
# 'Massachusetts',
# 'Mauritius',
# 'Mexico',
# 'Michigan',
# 'Minnesota',
# 'Moldova',
# 'Monaco',
# 'Montenegro',
# 'Morocco',
# 'Mozambique',
# 'Myanmar',
# 'México',
mappings['México'] = 'Mexico'
# 'Nebraska',
# 'Netherlands',
# 'Nevada',
# 'New Caledonia',
# 'New Hampshire',
# 'New York (State)',
# 'New Zealand',
# 'Nicaragua',
# 'Nigeria',
# 'No place, unknown, or undetermined',
# 'North Carolina',
# 'Noruega',
mappings['Noruega'] = 'Norway'
# 'Norway',
# 'Ohio',
# 'Ottoman Empire',
# 'Pakistan',
# 'Palestinian Territory, Occupied',
mappings['Palestinian Territory, Occupied'] = 'Palestine'
# 'Palestinian territories',
mappings['Palestinian territories'] = 'Palestine'
# 'Panama',
# 'Panamá',
mappings['Panamá'] = 'Panama'
# 'Paraguay',
# 'Países Bajos',
mappings['Países Bajos'] = 'Netherlands'
# 'Pennsylvania',
# 'Peru',
# 'Perú',
mappings['Perú'] = 'Peru'
# 'Philippines',
# 'Poland',
# 'Polonia',
mappings['Polonia'] = 'Poland'
# 'Portugal',
# 'Prussia',
# 'Puerto Rico',
# 'Reino Unido',
mappings['Reino Unido'] = 'United Kingdom'
# 'Republic of Macedonia',
# 'Republic of the Congo',
# 'República Checa',
mappings['República Checa'] = 'Czech Republic'
# 'República de Weimar',
mappings['República de Weimar'] = 'Weimar Republic'
# 'Romania',
# 'Rumania',
mappings['Rumania'] = 'Romania'
# 'Rusia',
mappings['Rusia'] = 'Russia'
# 'Russia',
# 'Saint Lucia',
# 'Saint-Martin',
# 'Scotland',
# 'Second Polish Republic',
mappings['Second Polish Republic'] = 'Poland'
# 'Senegal',
# 'Serbia',
# 'Sierra Leone',
# 'Sikkim',
# 'Singapore',
# 'Siria',
mappings['Siria'] = 'Syria'
# 'Slovakia',
# 'Slovenia',
# 'Somalia',
# 'South Africa',
# 'South Carolina',
# 'South Korea',
# 'Soviet Union',
mappings['Soviet Union'] = 'Russia'
# 'Spain',
# 'Sri Lanka',
# 'Sudan',
# 'Suecia',
mappings['Suecia'] = 'Sweden'
# 'Suiza',
mappings['Suiza'] = 'Switzerland'
# 'Swaziland',
# 'Sweden',
# 'Switzerland',
# 'Syria',
# 'Taiwan',
# 'Tajikistan',
# 'Tanzania',
# 'Tennessee',
# 'Thailand',
# 'The Bahamas',
# 'Trinidad and Tobago',
# 'Tunisia',
# 'Turkey',
# 'Turquía',
mappings['Turquía'] = 'Turkey'
# 'Túnez',
mappings['Túnez'] = 'Tunisia'
# 'Ucrania',
mappings['Ucrania'] = 'Ukraine'
# 'Uganda',
# 'Ukraine',
# 'United Kingdom',
# 'United States',
# 'Uruguay',
# 'Utah',
# 'Uzbekistan',
# 'Uzbekistán',
mappings['Uzbekistán'] = 'Uzbekistan'
# 'Venezuela',
# 'Vermont',
# 'Vietnam',
# 'Virginia',
# 'Washington (State)',
# 'Zambia',
# 'Zimbabwe',
# 'África Occidental Portuguesa'
mappings['África Occidental Portuguesa'] = 'Portuguese Angola'

## Correcting countries

In [33]:
%%time

def process_pais_ejemplar(x):
    c = x['PAIS_EJEMPLAR']
    return mappings.get(c, c)

autores_df['PAIS_EJEMPLAR'] = autores_df.apply(process_pais_ejemplar, axis=1)

CPU times: user 1.68 s, sys: 8.03 ms, total: 1.69 s
Wall time: 1.69 s


In [34]:
%%time

def process_pais_autor(x):
    c = x['PAIS_AUTOR']
    return mappings.get(c, c)

autores_df['PAIS_AUTOR'] = autores_df.apply(process_pais_autor, axis=1)

CPU times: user 1.69 s, sys: 3.97 ms, total: 1.69 s
Wall time: 1.69 s


In [35]:
autores_df['PAIS_EJEMPLAR'].nunique()

111

In [36]:
autores_df['PAIS_AUTOR'].nunique()

150

# Saving results

In [37]:
autores_df.to_csv('autores6.csv', index=False)